# Fashion Concierge – Interactive Demo Notebook

This notebook is an **interactive front end** for the Fashion Concierge project built with the
**Google Agent Development Kit (ADK)** and **Gemini**.

It is designed to work for **any user** who has access to the repository, whether they run it:

- in **GitHub Codespaces**, or  
- in a local Python environment (VS Code, Jupyter Lab, etc.).

The notebook demonstrates three core capabilities:

1. Wiring the notebook to the **Fashion Concierge backend** (the `FashionConciergeApp` class in `adk_app/app.py`).  
2. Running **one end-to-end outfit suggestion** through the agent pipeline.  
3. Showing a **simple session and memory example** where the agent remembers user preferences.


## 0. How to use this notebook

### 0.1 Open the notebook in GitHub Codespaces

1. Navigate to the Fashion Concierge repository on GitHub.  
2. Click **Code → Open with Codespaces → New codespace**.  
3. When the Codespace finishes starting, open this notebook file (for example `notebooks/fashion_concierge_demo.ipynb`) in the editor.  
4. Select the default Python kernel for the Codespace.

> You can also run this notebook locally if you prefer. The steps are the same after cloning the repo.

### 0.2 Install dependencies

Inside the Codespace (or your local terminal), run once:

```bash
pip install -r requirements.txt
# or, if the project is packaged
pip install -e .
```

### 0.3 Configure credentials

Make sure the environment has credentials for:

- Google Gemini / Vertex AI (for the LLM calls).  
- Any external tools you actually use (calendar, weather, etc.), if those are enabled.

In many setups this is handled via environment variables (`GOOGLE_API_KEY`, `GOOGLE_CLOUD_PROJECT`, etc.).
The next section will show where these are read.


## 1. Environment configuration

In [ ]:
"""Environment and path setup.

This cell:

1. Locates the repository root (so imports work whether the notebook lives in
   the root or in a subfolder like `notebooks/`).
2. Adds the repo root to `sys.path`.
3. Sets **placeholder** environment variables for Google Cloud / Gemini so that
   the rest of the code can read them. In Codespaces or local dev you should
   either:
   - export real values before starting Jupyter, or
   - edit the placeholders below.
"""

import os
import sys
from pathlib import Path

# --- Locate the project root --------------------------------------------------
# We look upwards from the notebook directory until we find the marker folder
# `adk_app` (which exists in this repo) or a `.git` directory.
current = Path().resolve()
project_root = None

for parent in [current] + list(current.parents):
    if (parent / "adk_app").exists() or (parent / ".git").exists():
        project_root = parent
        break

if project_root is None:
    raise RuntimeError(
        "Could not locate the project root. Make sure you are running this "
        "notebook inside the Fashion Concierge repository."
    )

print("Detected project root:", project_root)

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Sanity check
if not (project_root / "adk_app").exists():
    raise RuntimeError(
        "The 'adk_app' package was not found at the detected project root. "
        "Please confirm the repo layout."
    )
else:
    print("Found 'adk_app' package. Imports should work.")

# --- Environment variables for Google / tools ---------------------------------
# These are **placeholders**. Replace with real values or rely on values
# already exported in the environment. Using `setdefault` means they will
# not override anything you configured outside the notebook.
os.environ.setdefault("GOOGLE_CLOUD_PROJECT", "YOUR_GCP_PROJECT_ID_HERE")
os.environ.setdefault("GOOGLE_CLOUD_LOCATION", "YOUR_GCP_REGION_HERE")  # e.g. "europe-west4"
os.environ.setdefault("GOOGLE_API_KEY", "YOUR_GEMINI_API_KEY_HERE")

print("GOOGLE_CLOUD_PROJECT:", os.environ.get("GOOGLE_CLOUD_PROJECT"))
print("GOOGLE_CLOUD_LOCATION:", os.environ.get("GOOGLE_CLOUD_LOCATION"))

## 2. Load the Fashion Concierge backend

The Fashion Concierge backend is wrapped in a convenience class `FashionConciergeApp`
defined in `adk_app/app.py`.

This class is responsible for:

- Constructing the ADK `App` object.  
- Registering all agents (calendar, weather, wardrobe, stylist, critic, etc.).  
- Exposing high-level methods for:
  - creating sessions  
  - orchestrating outfit planning  
  - running conversations

We import and instantiate it here.

> If your repository uses a different entry point (for example a `create_app()`
> function), you only need to adapt the import below.


In [ ]:
from adk_app.app import FashionConciergeApp  # adapt if your module name differs

fashion_app = FashionConciergeApp()

print("FashionConciergeApp instance:", fashion_app)
print("Public attributes:", [a for a in dir(fashion_app) if not a.startswith("_")])

### 2.1 Helper functions for sessions and outfit orchestration

The HTTP API layer (in `server/api.py`) usually calls methods on `FashionConciergeApp`
to:

- create a new session, and  
- orchestrate an outfit recommendation.

To make the notebook easier to read, we mirror that behaviour with two helper
functions:

- `create_demo_session(...)`  
- `orchestrate_outfit(...)`  

If the method names in your implementation differ, you can adjust **only this
section** and the rest of the notebook will still work.


In [ ]:
from typing import Optional, Dict, Any, Any

def create_demo_session(metadata: Optional[Dict[str, Any]] = None):
    """Create a new session through `FashionConciergeApp`.

    Expected backend method:
        fashion_app.create_session(metadata: dict | None) -> Session

    If your app exposes a different method name, update this function.
    """
    if not hasattr(fashion_app, "create_session"):
        raise AttributeError(
            "FashionConciergeApp has no 'create_session' method. "
            "Open adk_app/app.py and either expose one or adapt this helper."
        )

    session = fashion_app.create_session(metadata=metadata or {"source": "notebook-demo"})
    session_id = getattr(session, "session_id", None) or getattr(session, "id", None)

    print("Created session object:", session)
    print("Session id:", session_id)
    return session, session_id


def orchestrate_outfit(
    session_id: str,
    user_query: str,
    location: str = "Amsterdam, NL",
    date_iso: Optional[str] = None,
    mood: Optional[str] = None,
) -> Any:
    """Call the high-level outfit orchestration method.

    The exact method name may differ slightly between versions of the repo.
    Common options are:

    - `orchestrate_outfit`
    - `plan_outfit`
    - `run_outfit_orchestration`

    This helper tries those in order. If none exist, update this function to
    call the appropriate method on `fashion_app`.
    """
    candidate_method_names = ["orchestrate_outfit", "plan_outfit", "run_outfit_orchestration"]

    method = None
    for name in candidate_method_names:
        if hasattr(fashion_app, name):
            method = getattr(fashion_app, name)
            break

    if method is None:
        raise AttributeError(
            "Could not find any of the expected outfit orchestration methods on "
            "FashionConciergeApp. Please open adk_app/app.py and update the "
            "helper `orchestrate_outfit` accordingly."
        )

    response = method(
        session_id=session_id,
        user_query=user_query,
        location=location,
        date=date_iso,
        mood=mood,
    )
    return response

## 3. End-to-end outfit suggestion demo

This section runs a **single full interaction**:

1. Create a fresh session.  
2. Ask the agent to plan outfits for a given day, location and mood.  
3. Inspect the structured result and any natural language rationale.

If your backend is wired as intended, this will exercise:

- the **calendar agent** (to infer formality and schedule),  
- the **weather agent** (to choose layers and fabrics), and  
- the **wardrobe / stylist agents** (to assemble and score outfit combinations).


In [ ]:
# 1. Create a new session for this demo
session, session_id = create_demo_session(metadata={"demo": "single-day-outfit"})

# 2. Define the high-level request for the orchestrator
user_query = (
    "I am in Amsterdam this Friday with a full workday and casual drinks after. "
    "Suggest one daytime outfit and one outfit that can transition into the evening. "
    "Keep the mood 'trendy' but practical for commuting by bike."
)

# Optional: you can specify a particular date if your backend expects it, e.g. "2025-11-28"
demo_date = None  # or "2025-11-28"

# 3. Call the orchestrator
outfit_response = orchestrate_outfit(
    session_id=session_id,
    user_query=user_query,
    location="Amsterdam, NL",
    date_iso=demo_date,
    mood="trendy",
)

outfit_response

### 3.1 Interpreting the result

The variable `outfit_response` may be:

- a plain string (just text),  
- a Pydantic model / dataclass, or  
- a nested dictionary with keys like `outfits`, `candidates`, `context`, etc.

For inspection and debugging it helps to **look at the raw object** and then
pull out key parts (for example, outfit items and their scores). The helper
below tries to do this in a best-effort way.


In [ ]:
from pprint import pprint

def pretty_print_outfits(resp):
    """Best-effort pretty printer for common response shapes."""
    if resp is None:
        print("No response returned.")
        return

    obj = resp
    if hasattr(resp, "model_dump"):
        obj = resp.model_dump()
    elif hasattr(resp, "dict"):
        obj = resp.dict()

    if isinstance(obj, dict):
        print("Top-level keys:", list(obj.keys()))
        for key in ("outfits", "candidates", "suggestions"):
            if key in obj:
                print("\n===", key, "===")
                pprint(obj[key], depth=3)
                break
        else:
            pprint(obj, depth=3)
    else:
        print(obj)


pretty_print_outfits(outfit_response)

## 4. Session and memory demo

Now we demonstrate how sessions and memory work together.

1. Start a **new session**.  
2. First turn: the user describes a long-term style preference.  
3. Second turn: the user asks for an outfit; the agent should incorporate that preference.

This relies on your backend wiring a conversational entry point that:

- keeps **short-term state** in the ADK session, and  
- optionally writes **long-term preferences** into a memory store.


In [ ]:
# Create a dedicated session for the memory demo
memory_session, memory_session_id = create_demo_session(metadata={"demo": "memory"})

# Turn 1 – the user states a preference
pref_query = "Remember that I prefer monochrome outfits with chunky sneakers and no logos."

# Many backends expose a generic conversation method, for example `run_conversation`.
# If your `FashionConciergeApp` uses a different name (e.g. `chat` or `orchestrate_chat`),
# adjust the call below.
if hasattr(fashion_app, "run_conversation"):
    first_turn = fashion_app.run_conversation(
        session_id=memory_session_id,
        user_query=pref_query,
    )
else:
    raise AttributeError(
        "This notebook expects a high-level conversation method named "
        "'run_conversation' on FashionConciergeApp. "
        "Update this cell to call whichever method your app uses."
    )

print("First turn response:\n", first_turn, "\n")

In [ ]:
# Turn 2 – ask for a casual outfit; the agent should remember the preference.

followup_query = "Now suggest an outfit for a relaxed Sunday brunch in Amsterdam."

second_turn = fashion_app.run_conversation(
    session_id=memory_session_id,
    user_query=followup_query,
)

print("Second turn response:\n", second_turn)

## 5. Where to go next

Once the basic flows above are working, you can extend this notebook to:

- Visualise **agent traces** (tool calls, reasoning steps) for a given session.  
- Call your **evaluation harness** to run predefined scenarios and compute scores.  
- Show how this notebook maps to the **HTTP API** in `server/api.py`, and how the same
  patterns apply when deploying to **Vertex AI Agent Engine** or **Cloud Run**.

Because this notebook is designed to run from inside the repository (including
GitHub Codespaces), you can commit it to version control and treat it as both:

- a **developer tool** for experimenting with the agent, and  
- a **readable artifact** for your capstone submission.
